In [77]:
using DataFrames, RDatasets, FixedEffectModels
using CSV, HTTP
using CuArrays

┌ Info: Precompiling CuArrays [3a865a2d-5b23-5a0f-bc46-62713ec82fae]
└ @ Base loading.jl:1278


In [35]:
# This is the basic Fixed effects example. It has like he formula approch like 
# in python moduals....
df = dataset("plm", "Cigar")
reg(df, @formula(log(Sales) ~ NDI + fe(State) + fe(Year)), Vcov.cluster(:State), weights = :Pop)

                             Fixed Effect Model                             
Number of obs:                   1380   Degrees of freedom:                32
R2:                             0.803   R2 Adjusted:                    0.799
F-Stat:                        11.615   p-value:                        0.001
R2 within:                      0.140   Iterations:                         5
log(Sales) |    Estimate  Std.Error  t value Pr(>|t|)   Lower 95%   Upper 95%
-----------------------------------------------------------------------------
NDI        | -4.29493e-5 1.26022e-5 -3.40808    0.001 -6.76713e-5 -1.82273e-5


In [ ]:
# I have a gravity dataset setup on my github site. So first you have to grab it 
# with the HTTP package

f = HTTP.get("https://raw.githubusercontent.com/mwaugh0328/Gravity-Estimation/master/gravity_data.csv").body;

# Then this converts the bytes to a "csv" file

csv = CSV.File(f);

# Then this creates the dataframe from the csv file

df = DataFrame(csv);

# this final bit then renames the columns based on there location. Not 
# as slick as pandas 

rename!(df, [1 => :importer, 2 => :exporter, 3 => :trade_data,
        4 => :distance, 5 => :border])

In [57]:
# This does some filtering drop the one values
# drop the zero trade observations. 
# note the ! which automatically changes the df

filter!(row -> row.trade_data != 1.0, df);

filter!(row -> row.trade_data != 0.0, df);

In [85]:
outreg = reg(df, @formula(log(trade_data) ~ fe(importer) + fe(exporter) + log(distance) + border), save = true)

                            Fixed Effect Model                            
Number of obs:                   866  Degrees of freedom:               62
R2:                            0.876  R2 Adjusted:                   0.866
F-Stat:                      528.187  p-value:                       0.000
R2 within:                     0.568  Iterations:                        5
log(trade_data) |  Estimate Std.Error t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------------------
log(distance)   | -0.992491 0.0383587 -25.874    0.000  -1.06779 -0.917197
border          |  0.301892  0.112216 2.69028    0.007 0.0816217  0.522163


In [86]:
# then the outreg.fe is a dataframe with the fixed effects lined
# up with the names. This is cool. Easy now todo model stuff with it.
# must have save = true to do this

first(outreg.fe, 10)

,importer,exporter,fe_importer,fe_exporter
,Int64,Int64,Float64?,Float64?
1,1,2,2.64295,-0.326149
2,1,3,2.64295,-0.817882
3,1,4,2.64295,0.802891
4,1,5,2.64295,0.434928
5,1,6,2.64295,-0.109777
6,1,7,2.64295,2.19209
7,1,8,2.64295,-1.5925
8,1,9,2.64295,-0.824852
9,1,10,2.64295,0.867375
